In [1]:
import cv2
import os
from tqdm import tqdm
import albumentations as A
from albumentations import BboxParams, Rotate
from albumentations.core.composition import BboxParams

In [2]:
def load_image_and_label(image_path, label_path):
  """
  Loads an image and its corresponding YOLO labels from a text file.

  Args:
    image_path: Path to the image file.
    label_path: Path to the text file containing YOLO labels.

  Returns:
    image: Loaded image as a NumPy array.
    labels: List of bounding boxes in YOLO format (class_id, x_min, y_min, x_max, y_max).
  """
  image = cv2.imread(image_path)
  labels = []
  with open(label_path, "r") as f:
    for line in f.readlines():
      data = line.strip().split(" ")
      class_id = int(data[0])
      x_center, y_center, bbox_width, bbox_height = float(data[1]), float(data[2]), float(data[3]), float(data[4])
      labels.append((class_id, x_center, y_center, bbox_width, bbox_height))
  return image, labels

In [8]:
import albumentations as A
from albumentations.core.composition import BboxParams

def augment_data(image_dir, label_dir, output_image_dir, output_label_dir, num_augmentations):
    # Define bbox_params with label_fields
    bbox_params = BboxParams(format='yolo', label_fields=['category_ids'])

    augmenter = A.Compose([
        #A.HorizontalFlip(p=1),
        #A.VerticalFlip(p=1)
        A.Blur(blur_limit=3, p=1.0)
        #A.RandomBrightnessContrast(p=1)
        #A.Rotate(45),
        #A.RandomCrop(p=1),
    ], bbox_params=bbox_params)

    for filename in tqdm(os.listdir(image_dir)):
        if filename.endswith(".jpg") or filename.endswith(".png"):
            image_path = os.path.join(image_dir, filename)
            label_path = os.path.join(label_dir, os.path.splitext(filename)[0] + ".txt")
            image, labels = load_image_and_label(image_path, label_path)

            for i in range(num_augmentations):
                bbox_coords = [label[1:] for label in labels]
                category_ids = [label[0] for label in labels]
                # Adjust labels to include category_ids at the end of each bounding box definition
                augmented = augmenter(image=image, bboxes=bbox_coords, category_ids=category_ids)

                augmented_image = augmented['image']
                augmented_bbox_coords = augmented['bboxes']
                augmented_labels = [[category_ids[idx], *box] for idx, box in enumerate(augmented_bbox_coords)]

                # combined_labels = list(zip([label[0] for label in labels], augmented_labels))
                # print(f"combined_labels: {combined_labels}")
                # class_id, bbox = combined_labels
                # final_label = [class_id] + list(bbox)
                # #print(f"image: {augmented_image}")
                print(f"Final label: {augmented_labels}")
                # Create output filenames with augmentation count
                output_image_filename = f"{os.path.splitext(filename)[0]}_augmented_{i}.jpg"
                output_label_filename = f"{os.path.splitext(filename)[0]}_augmented_{i}.txt"

                # Save augmented image and labels
                cv2.imwrite(os.path.join(output_image_dir, output_image_filename), augmented_image)
                with open(os.path.join(output_label_dir, output_label_filename), "w") as f:
                    for label in augmented_labels:
                        f.write(f"{label[0]} {label[1]} {label[2]} {label[3]} {label[4]}\n")


In [9]:
if __name__ == "__main__":
  image_dir = "images"  
  label_dir = "labels" 
  output_image_dir = "augmented_images" 
  output_label_dir = "augmented_labels"  
  num_augmentations = 1

  # Create output directories if they don't exist
  os.makedirs(output_image_dir, exist_ok=True)
  os.makedirs(output_label_dir, exist_ok=True)

  augment_data(image_dir, label_dir, output_image_dir, output_label_dir, num_augmentations)

  print(f"Successfully created {num_augmentations * len(os.listdir(image_dir))} augmented images.")

100%|██████████| 2/2 [00:00<00:00, 11.84it/s]

Final label: [[1, 0.5475539999999999, 0.327087, 0.13295899999999994, 0.170358], [0, 0.54396, 0.600511, 0.14254200000000006, 0.19931900000000002]]
Final label: [[2, 0.450483, 0.540886, 0.37251599999999996, 0.26746200000000003]]
Successfully created 2 augmented images.
